In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
import time
import datetime
import yfinance as yf
from pretty_html_table import build_table
import smtplib
import email.message

### Acessando o site

In [2]:
driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()))
url = "https://www.investsite.com.br/seleciona_acoes.php"
driver.get(url)

### Clicando no botão "Procurar Ações"

In [3]:
botao_procurar = driver.find_element('xpath', '/html/body/div[1]/div[3]/div/form/input[1]')
driver.execute_script("arguments[0].click();", botao_procurar)

### Na tag < select > escolher "Mostrar Todos por página"

In [4]:
select = Select(driver.find_element('xpath', '/html/body/div[1]/div[3]/div[2]/div[1]/div[1]/select'))
select = select.select_by_value('-1')

### Salvando a tabela html para csv

In [5]:
tabela = driver.find_element("xpath", '/html/body/div[1]/div[3]/div[2]/div[2]/div[2]/table')
html_tabela = tabela.get_attribute("outerHTML")
data = pd.read_html(html_tabela)[0]
data.to_csv('screener.csv')

### Fechando o chrome

In [6]:
driver.quit()

### Tratando os dados e aplicando os filtros
Filtros:
<ul>
    <li>Volume Financeiro > R$100.000,00</li>
    <li>EV/EBIT positivo</li>
    <li>P/VPA positivo</li>
    <li>ROIC positivo</li>
<ul>


In [7]:
#abrindo a planilha
screener = pd.read_csv('screener.csv')
#nomeando colunas
screener.rename(columns={'Unnamed: 0': 'ticker',
                        'Unnamed: 3': 'roic',
                        'Unnamed: 9': 'pvpa',
                        'Unnamed: 10': 'evebit',
                        'Unnamed: 12': 'volfin'},inplace=True)
#excluindo colunas que não serão utilizadas
screener = screener.drop(['Unnamed: 0.1', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 4', 'Unnamed: 5',
                        'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 11' ,'Unnamed: 13'], axis=1)


#filtrando volume financeiro > 100k
screener['volfin'] = screener['volfin'].str.replace('.', '')
screener['volfin'] = pd.to_numeric(screener['volfin'], errors='coerce')
screener = screener.loc[screener['volfin'] > 100000]

#filtrando ev/ebit > 0
screener['evebit'] = screener['evebit'].str.replace('.', '')
screener['evebit'] = pd.to_numeric(screener['evebit'], errors='coerce')
screener = screener.loc[screener['evebit'] > 0]
screener['evebit'] = screener['evebit'] / 100

#filtrando p/vpa > 0
screener['pvpa'] = pd.to_numeric(screener['pvpa'], errors='coerce')
screener['pvpa'] = screener['pvpa'] / 100
screener = screener.loc[screener['pvpa'] > 0]

#filtrando roic > 0
screener['roic'] = screener['roic'].str.replace('%', '')
screener['roic'] = screener['roic'].str.replace(',', '')
screener['roic'] = pd.to_numeric(screener['roic'], errors='coerce')
screener['roic'] = screener['roic'] / 100
screener = screener.loc[screener['roic'] > 0]

#salvando
screener.to_csv('formula.csv')


/tmp/ipykernel_25092/2667562217.py:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  screener['volfin'] = screener['volfin'].str.replace('.', '')
/tmp/ipykernel_25092/2667562217.py:20: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  screener['evebit'] = screener['evebit'].str.replace('.', '')


### Salvando a tabela formatada para csv

In [8]:
formula = pd.read_csv('formula.csv')
formula = formula.drop(['Unnamed: 0', 'volfin'], axis=1)

### Ranking de ROIC

In [9]:
formula.sort_values(by='roic', ascending=False, inplace=True) 
formula.to_csv('roic.csv')
roic = pd.read_csv('roic.csv')
roic['rankingroic'] = roic.index
roic = roic.drop(columns=['Unnamed: 0', 'pvpa', 'evebit'])

### Ranking de EV/EBIT

In [10]:
formula.sort_values(by='evebit', ascending=True, inplace=True) 
formula.to_csv('evebit.csv')
evebit = pd.read_csv('evebit.csv')
evebit['rankingevebit'] = evebit.index
evebit = evebit.drop(columns=['Unnamed: 0', 'pvpa', 'roic'])

### Cruzando os ranking de ROIC x EV/EBIT para criar o ranking de pontuação da fórmula

In [11]:
#filtrando as empresas nos rankings
evebit_sorted = evebit.sort_values(['ticker'], ascending=[True])
roic_sorted = roic.sort_values(['ticker'], ascending=[True])

#somando os rankings para criar a fórmula
ranking = evebit_sorted.merge(roic_sorted, on='ticker')
ranking['ranking'] = ranking['rankingevebit'] + ranking['rankingroic']
ranking = ranking.sort_values(['ranking'], ascending=[True])
ranking = ranking.drop(columns=['rankingevebit', 'rankingroic'])

#excluindo empresas duplicadas da lista
ranking = ranking[~ranking.ticker.isin(['BRAP3', 'TASA3', 'PETR3', 'GOAU3', 'GOAU4', 'GGBR3', 'BRKM3', 'RNEW3', 'RNEW11', 'UNIP3', 'UNIP5', 'USIM3', 'DEXP4'])]

ranking = ranking.reset_index(drop=True)
#variavel para salvar a ordem do ranking
ordem = ranking.head(20)
ranking = ordem
ranking['ticker'] = ranking['ticker'].astype(str) + '.SA'
ranking = ranking["ticker"].tolist()
ordem['ticker'] = ordem['ticker'].str.slice(stop=-3)


### Puxando as cotações do Yahoo Finance

In [12]:
hoje = datetime.datetime.now()
um_ano_atras = hoje - datetime.timedelta(days = 365)
dados_mercado = yf.download(ranking, um_ano_atras, hoje)
dados_mercado = dados_mercado['Close']

[*********************100%***********************]  20 of 20 completed


### Calculando as variações

In [13]:
retorno_30d = ((dados_mercado.iloc[-2, :] / dados_mercado.iloc[-21, :]) - 1)
retorno_30d = round(retorno_30d * 100, 2)

retorno_180 = ((dados_mercado.iloc[-2, :] / dados_mercado.iloc[-123, :]) - 1)
retorno_180 = round(retorno_180 * 100, 2)

retorno_365d = ((dados_mercado.iloc[-2, :] / dados_mercado.iloc[0, :]) - 1)
retorno_365d = round(retorno_365d * 100, 2)

retorno = pd.concat([retorno_30d, retorno_180, retorno_365d], axis=1)

retorno.rename(columns={0: '30d (%)',
                        1: '180d (%)',
                        2: '365d (%)'},inplace=True)

#tirando o .SA do final do nome das empresas
retorno = retorno.reset_index(drop=False)
retorno.rename(columns={'index': 'ticker'}, inplace=True)
retorno = retorno.astype({'ticker':'string'})
retorno['ticker'] = retorno['ticker'].str.slice(stop=-3)

#reordenando conforme o ranking da formula

retorno = pd.merge(retorno, ordem, on='ticker')
retorno = retorno.sort_values('ranking', ascending=True)

retorno = retorno.reset_index(drop=True)
#criando a coluna Ranking
retorno['Ranking'] = range(1, len(retorno)+1)

#renomeando as colunas
retorno = retorno.rename(columns={'ticker': 'Empresa', 'evebit': 'EV/EBIT', 'roic': 'ROIC (%)', 'ranking': 'Pontos'})

#reordenando as colunas
new_col_order = ['Ranking', 'Pontos', 'Empresa', 'EV/EBIT', 'ROIC (%)', '30d (%)', '180d (%)', '365d (%)']
retorno = retorno.reindex(columns=new_col_order)

retorno['Ranking'] = retorno['Ranking'].astype(str) + '°'


### Transformando o dataframe em tabela HTML e estilizando

In [14]:
html_table = build_table(retorno, 'blue_light')

### Declarando a função para enviar email

In [15]:
def enviar_email(pessoas):
    corpo_email = f'''
    {html_table}
    <p>A <b>"Magic Formula"</b> é uma técnica introduzida pelo investidor Joel Greenblatt que consiste na criação de um ranking com as melhores empresas da bolsa de acordo com dois critérios, sendo um de valor (EV/EBIT) e outro de rentabilidade (ROIC). O ranking final é a soma de suas pontuações em cada critério.</p>
    <p>A tabela acima foi criada a partir de um banco de dados da bolsa brasileira, onde todas as empresas foram analisadas com os valores do último fechamento de mercado.</p>
    <p>Primeiramente é feito um <b>filtro de qualidade</b> excluindo as empresas que não atendem os seguintes critérios:<br>
    • Volume Financeiro > R$100.000,00<br>
    • EV/EBIT positivo<br>
    • P/VPA positivo<br>
    • ROIC positivo</p>
    
    <em><b>EV/EBIT:</b> Enterprise Value (Valor da Firma) dividido por Earnings Before Interest and Taxes (Lucro Operacional).</em><br>
    <em><b>ROIC:</b> Return Over Invested Capital (Retorno sobre o Capital Investido).</em><br>
    <em><b>Volume Financeiro:</b> Volume financeiro médio transacionado diariamente (média dos últimos 90 dias).</em><br>
    <em><b>P/VPA:</b> Preço dividido pelo Valor Patrimonial.</em>
    <p><b>***NÃO É UMA RECOMENDAÇÃO DE INVESTIMENTO***</b></p>
    '''
    msg = email.message.Message()
    msg['Subject'] = "Magic Formula B3"

    #seu email
    msg['From'] = "seu email aqui"
    msg['To'] = pessoas

    #senha de app do gmail
    password = 'sua senha aqui'
    msg.add_header('Content-Type', 'text/html')
    msg.set_payload(corpo_email)
     

    s = smtplib.SMTP('smtp.gmail.com: 587')
    s.starttls()
    s.login(msg['From'], password)
    s.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))
    print('Email enviado')

### Chamando a função de enviar email

In [16]:
#enviar_email()

### Função para enviar o email para vários destinatários

In [18]:
pessoas = ["seu email aqui"]
for pessoa in pessoas:
    enviar_email(pessoa)

Email enviado
